In [1]:
import requests
from io import StringIO
import pandas as pd
import re
import datetime
import time

## 起始日期
d = int(input('要抓幾天前的資料勒？(當天=0): '))
today = datetime.datetime.now() - datetime.timedelta(days=d)
today_folder_name = str(today)[0:10]
print('請確認起始日期：' + today_folder_name)

## 抓起始日回推 15 日的買超資料
TWSE_SITB = {}
TPEX_SITB = {}
for i in range(15):
    date = today - datetime.timedelta(days=i)
    date0 = str(date)[:10].split('-')
    
    #=====================================上市公司====================================#
    date1 = str(date0[0]) + str(date0[1]) + str(date0[2])
    r = requests.get('http://www.twse.com.tw/fund/T86?response=csv&date='+date1+'&selectType=ALLBUT0999')#
    lines = r.text.strip().replace(' ','').replace('\r\n','')
    LL = re.split('","|",="',lines)[19:-1]
    
    for j in range(len(LL)):
        if '\u4e00' <= LL[j] <= '\u9fa5' and len(LL[j-1]) == 4:
            code = LL[j-1]
            volume = int(LL[j+9].replace(',',''))/1000
            if volume != 0:
                if not code in list(TWSE_SITB.keys()):
                    TWSE_SITB[code] = [0]*i + [volume]
                else:
                    TWSE_SITB[code] = TWSE_SITB[code] + [volume]
    
    SITB_list = list(TWSE_SITB.keys())
    for j in SITB_list:
         if len(TWSE_SITB[j]) < i+1:
            TWSE_SITB[j] = TWSE_SITB[j] + [0]
                        
    print(date1 + ' done!')                    
    time.sleep(2)
    #================================================================================#
    
    #====================================上櫃公司=====================================#
    date2 = str(int(date0[0])-1911) +'/'+ str(date0[1]) +'/'+ str(date0[2])
    r = requests.get('https://www.tpex.org.tw/web/stock/3insti/daily_trade/3itrade_hedge_result.php?l=zh-tw&o=csv&se=EW&t=D&d=' + date2 + '&s=0')
    lines = r.text.strip().replace(' ','').replace('\r\n','').replace(',"','').replace('""','"')
    LL = lines.split('"')[1:-1]    
    
    for j in range(len(LL)):
        if j%24 == 0 and len(LL[j]) == 4:
            code = LL[j]
            volume = int(LL[j+13].replace(',',''))/1000
            if volume != 0:
                if not code in list(TPEX_SITB.keys()):
                    TPEX_SITB[code] = [0]*i + [volume]
                else:
                    TPEX_SITB[code] = TPEX_SITB[code] + [volume]
    
    SITB_list = list(TPEX_SITB.keys())
    for j in SITB_list:
         if len(TPEX_SITB[j]) < i+1:
            TPEX_SITB[j] = TPEX_SITB[j] + [0]
    
    print(date2 + ' done!')                    
    time.sleep(2)
    #=================================================================================#
    
# print(TWSE_SITB)
# print(TPEX_SITB)

要抓幾天前的資料勒？(當天=0): 0
請確認起始日期：2023-01-27
20230127 done!
112/01/27 done!
20230126 done!
112/01/26 done!
20230125 done!
112/01/25 done!
20230124 done!
112/01/24 done!
20230123 done!
112/01/23 done!
20230122 done!
112/01/22 done!
20230121 done!
112/01/21 done!
20230120 done!
112/01/20 done!
20230119 done!
112/01/19 done!
20230118 done!
112/01/18 done!
20230117 done!
112/01/17 done!
20230116 done!
112/01/16 done!
20230115 done!
112/01/15 done!
20230114 done!
112/01/14 done!
20230113 done!
112/01/13 done!


In [2]:
## 根據投信買賣超給分，並排序
#=====================================上市公司====================================#
TWSE_SITB_score = {}
codes = list(TWSE_SITB.keys())
SITBs = list(TWSE_SITB.values())
for i in range(len(codes)):
    code = codes[i]
    SITB = SITBs[i]
    score = 0
    
    ## 計分
    for j in range(len(SITB)):
        if SITB[j] > 0:
            score += (len(SITB) - j)
            
    ## 近15天總買超量少於100張，則給-1分
    BT = 0
    for j in range(len(SITB)):
        if SITB[j] > 0:
            BT = BT + SITB[j]
    if BT < 100:
        score = -1
            
    ## 近3天無買超，且近3天賣超總量大於近15天最大單日買超量，則給-1分
    if max(SITB[0:3]) <= 0 and max(SITB)-abs(sum(SITB[0:3])) < 0:
        score = -1
    
    ## 近期最大賣超量大於近期最大買超量，且近期偏向賣超，則給-1分
    if max(SITB) < abs(min(SITB)) and SITB.index(max(SITB)) > SITB.index(min(SITB)):
        score = -1
        
    ## 近5天合計為賣超，則給-1分
    if sum(SITB[0:5]) < 0:
        score = -1
    
    ## 若為金融股，則給-1分
    if code[:2] == '28':
        score = -1
        
    if score > 0:
        TWSE_SITB_score[code] = score
        
T = sorted(TWSE_SITB_score.items(), key=lambda x:x[1],reverse=True)
TWSE_SITB_rank = []
for i in T:
    code = i[0]
    TWSE_SITB_rank.append(code)
#================================================================================#

#====================================上櫃公司=====================================#
TPEX_SITB_score = {}
codes = list(TPEX_SITB.keys())
SITBs = list(TPEX_SITB.values())
for i in range(len(codes)):
    code = codes[i]
    SITB = SITBs[i]
    score = 0
    
    ## 計分
    for j in range(len(SITB)):
        if SITB[j] > 0:
            score += (len(SITB) - j)
    
    ## 近15天總買超量少於100張，則給-1分
    BT = 0
    for j in range(len(SITB)):
        if SITB[j] > 0:
            BT = BT + SITB[j]
    if BT < 100:
        score = -1
    
    ## 近3天無買超，且近3天賣超總量大於近15天最大單日買超量，則給-1分
    if max(SITB[0:3]) <= 0 and max(SITB)-abs(sum(SITB[0:3])) < 0:
        score = -1
    
    ## 近期最大賣超量大於近期最大買超量，且近期偏向賣超，則給-1分
    if max(SITB) < abs(min(SITB)) and SITB.index(max(SITB)) > SITB.index(min(SITB)):
        score = -1
    
    ## 近5天合計為賣超，則給-1分
    if sum(SITB[0:5]) < 0:
        score = -1
    
    ## 若為金融股，則給-1分
    if code[:2] == '58' or code[:2] == '60':
        score = -1
        
    if score > 0:
        TPEX_SITB_score[code] = score
        
T = sorted(TPEX_SITB_score.items(), key=lambda x:x[1],reverse=True)
TPEX_SITB_rank = []
for i in T:
    code = i[0]
    TPEX_SITB_rank.append(code)
#=================================================================================#

In [3]:
## 儲存上市上櫃全部SITB_rank成果到電腦
import os
import numpy as np
## 回到原始路徑
# os.chdir('D:\\WEI\\TWStock')
patho = os.getcwd() 
os.chdir(os.getcwd() + '\SITB_rank')

pathw = os.getcwd() + '\\' + str(today_folder_name)
if not os.path.isdir(pathw):
    os.mkdir(pathw)

os.chdir(pathw)
np.save('TWSE_SITB_rank',TWSE_SITB_rank)
np.save('TPEX_SITB_rank',TPEX_SITB_rank)
os.chdir(patho)

In [4]:
print(TPEX_SITB_score)

{'2732': 9, '3218': 9, '3624': 10, '3675': 10, '5425': 5, '6146': 10, '6147': 10, '6245': 10, '6274': 10, '6472': 5, '6488': 10, '6679': 5, '6741': 6, '8044': 5, '8069': 10, '8086': 5, '8155': 9, '8358': 10}
